In [51]:
import requests
import pysb
import time
from IPython.display import display
import getpass

from usgs_datatools import doi
from usgs_datatools.doi import add_primary_doi_manager

In [3]:
_rangeMapRoot = "5951527de4b062508e3b1e79"

In [4]:
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: sbristol@usgs.gov
········
Invalid password, try again
········


In [52]:
username = 'sbristol@usgs.gov'
password = getpass.getpass('USGS AD Password: ')

USGS AD Password: ········


In [53]:
DoiSession = doi.DoiSession(env='staging')

Using Staging Environment



In [54]:
DoiSession.doi_authenticate(username, password)

In [55]:
rangeMaps = {}
rangeMapDocs = []
rangeMapSearchResults = sb.find_items("parentId="+_rangeMapRoot+"&fields=title,body,contacts,identifiers&max=100")
while rangeMapSearchResults is not None:
    for rangeItem in rangeMapSearchResults["items"]:
        rangeMaps[next((i for i in rangeItem["identifiers"] if i["type"] == "GAP_SpeciesCode"), None)["key"]] = rangeItem["id"]
        thisRangeMapDoc = {}
        thisRangeMapDoc["id"] = rangeItem["id"]
        thisRangeMapDoc["identifiers"] = rangeItem["identifiers"]
        thisRangeMapDoc["title"] = rangeItem["title"]
        thisRangeMapDoc["body"] = rangeItem["body"]
        thisRangeMapDoc["contacts"] = rangeItem["contacts"]
        rangeMapDocs.append(thisRangeMapDoc)
    rangeMapSearchResults = sb.next(rangeMapSearchResults)

In [80]:
count = 0
for rangeItem in rangeMapDocs:
    if count == 2:
        break
    
    doiRegistration = {}
    doiRegistration["status"] = "reserved"
    doiRegistration["resourceURL"] = "https://www.sciencebase.gov/catalog/item/"+rangeItem["id"]
    doiRegistration["title"] = rangeItem["title"]
    doiRegistration["abstractTypeDesc"] = rangeItem["body"]
    doiRegistration["pubYear"] = "2017"
    doiRegistration["resourceType"] = "Dataset"
    for index,contact in enumerate(rangeItem["contacts"]):
        if contact["type"] == "reviewer":
            lastAuthor = contact
        else:
            doiRegistration["authors["+str(index - 1)+"].position"] = str(index - 1)
            doiRegistration["authors["+str(index - 1)+"].orcId"] = ""
            doiRegistration["authors["+str(index - 1)+"].authorName"] = contact["name"].split(" ")[-1]+", "+contact["name"].split(" ")[0]
            lastAuthorPosition = index - 1
    if lastAuthor is not None:
        doiRegistration["authors["+str(lastAuthorPosition + 1)+"].position"] = str(lastAuthorPosition + 1)
        doiRegistration["authors["+str(lastAuthorPosition + 1)+"].orcId"] = ""
        doiRegistration["authors["+str(lastAuthorPosition + 1)+"].authorName"] = lastAuthor["name"].split(" ")[-1]+", "+lastAuthor["name"].split(" ")[0]
        
    display (doiRegistration)
    print (DoiSession.doi_create(doiRegistration))
    count = count + 1


{'abstractTypeDesc': 'This dataset represents a species known range extent for Taricha torosa (California Newt). These range maps are created by attributing sub-watershed polygons with information of a species presence, origin, seasonal and reproductive use.',
 'authors[0].authorName': 'Boykin, Ken',
 'authors[0].orcId': '',
 'authors[0].position': '0',
 'authors[1].authorName': 'Lonneker, Jeff',
 'authors[1].orcId': '',
 'authors[1].position': '1',
 'pubYear': '2017',
 'resourceType': 'Dataset',
 'resourceURL': 'https://www.sciencebase.gov/catalog/item/59f5ebb0e4b063d5d307e0ad',
 'status': 'reserved',
 'title': 'Taricha torosa (California Newt) Range Map'}

IndexError: list index out of range